# 🔄 ChromaDB 동기화 시스템

이 노트북은 `raw_data` 폴더와 ChromaDB를 자동으로 동기화하는 전용 시스템입니다.

## 📋 주요 기능
1. **Raw Data 폴더 스캔**: .txt, .pdf 파일 자동 감지
2. **스마트 동기화**: 새 파일만 선별하여 DB 추가
3. **상태 모니터링**: 상세한 동기화 리포트
4. **고아 파일 감지**: DB에만 있는 파일 식별
5. **검색 테스트**: 동기화 후 검색 기능 확인

## ⚠️ 사용 전 준비사항
- `.env` 파일에 `OPENAI_API_KEY` 설정 필요
- `raw_data` 폴더에 동기화할 문서 파일 준비

## 🔧 라이브러리 설치 및 임포트

In [ ]:
# 필요한 패키지 설치 (처음 실행 시에만)
# !pip install chromadb langchain-openai langchain-community langchain-chroma python-dotenv pypdf

In [1]:
import os
import shutil
from pathlib import Path
import pandas as pd
from typing import List, Dict, Any

# Langchain imports
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 환경변수 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 모든 라이브러리가 성공적으로 임포트되었습니다.")

# API 키 확인
if os.getenv('OPENAI_API_KEY'):
    print("🔑 OpenAI API 키가 설정되어 있습니다.")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

✅ 모든 라이브러리가 성공적으로 임포트되었습니다.
🔑 OpenAI API 키가 설정되어 있습니다.


## 🛠️ ChromaDB 관리 클래스

In [2]:
class ChromaDBManager:
    """ChromaDB 관리를 위한 클래스"""
    
    def __init__(self, db_path: str = "./chroma_db", embedding_model: str = "text-embedding-3-large"):
        self.db_path = db_path
        self.embedding_model = embedding_model
        self.embeddings = OpenAIEmbeddings(model=embedding_model)
        self.db = None
        
        print(f"📁 DB 경로: {self.db_path}")
        print(f"🔤 임베딩 모델: {self.embedding_model}")
    
    def check_db_exists(self) -> bool:
        """DB 존재 여부 확인"""
        return os.path.exists(self.db_path) and os.path.isdir(self.db_path) and len(os.listdir(self.db_path)) > 0
    
    def create_new_db(self, documents: list, force_recreate: bool = False) -> bool:
        """새로운 ChromaDB 생성"""
        try:
            if force_recreate and self.check_db_exists():
                print("🗑️ 기존 DB를 삭제하고 새로 생성합니다...")
                self.delete_db()
            
            print(f"🔨 새로운 ChromaDB를 생성하고 {len(documents)}개 문서를 추가합니다...")
            
            self.db = Chroma.from_documents(
                documents=documents,
                embedding=self.embeddings,
                collection_name="rag_collection",
                persist_directory=self.db_path
            )
            
            print(f"✅ ChromaDB가 성공적으로 생성되었습니다! ({len(documents)}개 문서)")
            return True
            
        except Exception as e:
            print(f"❌ DB 생성 실패: {str(e)}")
            return False
    
    def load_existing_db(self) -> bool:
        """기존 ChromaDB 로드"""
        try:
            if not self.check_db_exists():
                print("❌ 로드할 DB가 존재하지 않습니다.")
                return False
            
            print("📥 기존 ChromaDB를 로드합니다...")
            
            self.db = Chroma(
                persist_directory=self.db_path,
                embedding_function=self.embeddings,
                collection_name="rag_collection"
            )
            
            count = self.get_document_count()
            print(f"✅ ChromaDB가 성공적으로 로드되었습니다! ({count}개 문서)")
            return True
            
        except Exception as e:
            print(f"❌ DB 로드 실패: {str(e)}")
            return False
    
    def add_documents(self, documents: list) -> bool:
        """기존 DB에 문서 추가"""
        try:
            if self.db is None:
                if not self.load_existing_db():
                    print("❌ DB를 로드할 수 없어 문서 추가를 실패했습니다.")
                    return False
            
            old_count = self.get_document_count()
            print(f"📝 기존 DB에 {len(documents)}개 문서를 추가합니다...")
            
            self.db.add_documents(documents)
            
            new_count = self.get_document_count()
            print(f"✅ 문서 추가 완료! ({old_count} → {new_count}개)")
            return True
            
        except Exception as e:
            print(f"❌ 문서 추가 실패: {str(e)}")
            return False
    
    def get_document_count(self) -> int:
        """저장된 문서 개수 반환"""
        try:
            if self.db is None:
                return 0
            return self.db._collection.count()
        except:
            return 0
    
    def get_files_in_db(self) -> List[str]:
        """DB에 저장된 파일명 목록 반환"""
        try:
            if self.db is None:
                return []
            
            collection = self.db._collection
            results = collection.get(include=['metadatas'])
            
            files_in_db = set()
            for metadata in results['metadatas']:
                if metadata and 'source' in metadata:
                    source = metadata['source']
                    filename = os.path.basename(source)
                    files_in_db.add(filename)
            
            return list(files_in_db)
            
        except Exception as e:
            print(f"❌ DB 파일 목록 조회 실패: {str(e)}")
            return []
    
    def delete_db(self) -> bool:
        """ChromaDB 완전 삭제"""
        try:
            self.db = None
            import gc
            gc.collect()
            
            if os.path.exists(self.db_path):
                shutil.rmtree(self.db_path)
                print(f"🗑️ ChromaDB가 완전히 삭제되었습니다: {self.db_path}")
            else:
                print("ℹ️ 삭제할 DB가 존재하지 않습니다.")
            
            return True
            
        except Exception as e:
            print(f"❌ DB 삭제 실패: {str(e)}")
            return False
    
    def get_status(self) -> Dict[str, Any]:
        """DB 상태 정보 반환"""
        exists = self.check_db_exists()
        loaded = self.db is not None
        count = self.get_document_count() if loaded else 0
        
        return {
            'db_exists': exists,
            'db_loaded': loaded,
            'document_count': count,
            'db_path': self.db_path,
            'embedding_model': self.embedding_model
        }

print("✅ ChromaDBManager 클래스가 정의되었습니다.")

✅ ChromaDBManager 클래스가 정의되었습니다.


## 📄 문서 처리 유틸리티 함수

In [3]:
def load_and_split_document(file_path: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list:
    """파일을 로드하고 청크로 분할"""
    try:
        file_extension = Path(file_path).suffix.lower()
        
        # 파일 타입에 따른 로더 선택
        if file_extension == '.txt':
            loader = TextLoader(file_path, encoding='utf-8')
        elif file_extension == '.pdf':
            loader = PyPDFLoader(file_path)
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {file_extension}")
        
        # 텍스트 분할기 설정
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        
        # 문서 로드 및 분할
        documents = loader.load_and_split(text_splitter)
        
        print(f"📄 파일 로드 성공: {os.path.basename(file_path)}")
        print(f"📊 생성된 청크 수: {len(documents)}")
        
        return documents
        
    except Exception as e:
        print(f"❌ 파일 로드 실패 ({os.path.basename(file_path)}): {str(e)}")
        return []

print("✅ 문서 처리 유틸리티 함수가 정의되었습니다.")

✅ 문서 처리 유틸리티 함수가 정의되었습니다.


## 🗂️ Raw Data 동기화 관리 클래스

In [4]:
class RawDataSyncManager:
    """Raw Data 폴더와 ChromaDB 동기화 관리 클래스"""
    
    def __init__(self, raw_data_path: str = "./raw_data"):
        self.raw_data_path = raw_data_path
        self.supported_extensions = ['.txt', '.pdf']
        
        # raw_data 폴더가 없으면 생성
        if not os.path.exists(self.raw_data_path):
            os.makedirs(self.raw_data_path)
            print(f"📁 raw_data 폴더를 생성했습니다: {self.raw_data_path}")
        else:
            print(f"📁 raw_data 폴더 경로: {self.raw_data_path}")
    
    def scan_raw_data_folder(self) -> List[Dict[str, Any]]:
        """raw_data 폴더의 모든 지원 파일 스캔"""
        files_info = []
        
        try:
            if not os.path.exists(self.raw_data_path):
                print(f"❌ raw_data 폴더가 존재하지 않습니다: {self.raw_data_path}")
                return files_info
            
            for root, dirs, files in os.walk(self.raw_data_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    file_extension = Path(file).suffix.lower()
                    
                    if file_extension in self.supported_extensions:
                        file_stat = os.stat(file_path)
                        
                        file_info = {
                            'filename': file,
                            'full_path': file_path,
                            'relative_path': os.path.relpath(file_path, self.raw_data_path),
                            'extension': file_extension,
                            'size_bytes': file_stat.st_size,
                            'size_mb': round(file_stat.st_size / (1024 * 1024), 2),
                            'modified_time': file_stat.st_mtime,
                            'modified_date': pd.to_datetime(file_stat.st_mtime, unit='s').strftime('%Y-%m-%d %H:%M:%S')
                        }
                        files_info.append(file_info)
            
            return files_info
            
        except Exception as e:
            print(f"❌ raw_data 폴더 스캔 실패: {str(e)}")
            return []
    
    def compare_with_db(self, db_manager: ChromaDBManager) -> Dict[str, List[str]]:
        """raw_data 폴더의 파일들과 DB에 저장된 파일들을 비교"""
        # raw_data 폴더 파일 목록
        raw_files_info = self.scan_raw_data_folder()
        raw_files = [info['filename'] for info in raw_files_info]
        
        # DB에 저장된 파일 목록
        db_files = db_manager.get_files_in_db()
        
        # 비교 결과
        sync_status = {
            'new_files': [],      # DB에 없는 새 파일들
            'existing_files': [], # DB에 이미 있는 파일들
            'orphaned_files': [], # raw_data에는 없지만 DB에 있는 파일들
            'all_raw_files': raw_files,
            'all_db_files': db_files
        }
        
        # 새 파일과 기존 파일 분류
        for filename in raw_files:
            if filename in db_files:
                sync_status['existing_files'].append(filename)
            else:
                sync_status['new_files'].append(filename)
        
        # 고아 파일 찾기 (DB에만 있고 raw_data에는 없는 파일)
        for filename in db_files:
            if filename not in raw_files:
                sync_status['orphaned_files'].append(filename)
        
        return sync_status
    
    def sync_with_db(self, db_manager: ChromaDBManager, chunk_size: int = 500, chunk_overlap: int = 100) -> bool:
        """raw_data 폴더의 새 파일들을 DB에 동기화"""
        try:
            # 동기화 상태 확인
            sync_status = self.compare_with_db(db_manager)
            new_files = sync_status['new_files']
            
            if not new_files:
                print("✅ 동기화할 새 파일이 없습니다. 모든 파일이 이미 DB에 저장되어 있습니다.")
                return True
            
            print(f"📝 {len(new_files)}개의 새 파일을 DB에 추가합니다...")
            
            # DB가 로드되지 않았다면 로드 시도
            if db_manager.db is None:
                if db_manager.check_db_exists():
                    if not db_manager.load_existing_db():
                        print("❌ 기존 DB 로드에 실패했습니다.")
                        return False
                else:
                    print("ℹ️ 기존 DB가 없습니다. 첫 번째 파일로 새 DB를 생성합니다.")
            
            # 각 새 파일 처리
            total_added_docs = 0
            for filename in new_files:
                file_path = os.path.join(self.raw_data_path, filename)
                
                print(f"\n🔄 처리 중: {filename}")
                
                # 파일 로드 및 분할
                documents = load_and_split_document(file_path, chunk_size, chunk_overlap)
                
                if documents:
                    # DB가 없으면 첫 번째 파일로 생성, 있으면 추가
                    if not db_manager.check_db_exists():
                        success = db_manager.create_new_db(documents)
                    else:
                        success = db_manager.add_documents(documents)
                    
                    if success:
                        total_added_docs += len(documents)
                        print(f"   ✅ {filename}: {len(documents)}개 청크 추가됨")
                    else:
                        print(f"   ❌ {filename}: 추가 실패")
                        return False
                else:
                    print(f"   ⚠️ {filename}: 파일 로드 실패")
            
            print(f"\n🎉 동기화 완료! 총 {total_added_docs}개 청크가 추가되었습니다.")
            return True
            
        except Exception as e:
            print(f"❌ 동기화 실패: {str(e)}")
            return False
    
    def print_sync_report(self, db_manager: ChromaDBManager):
        """동기화 상태 리포트 출력"""
        print("\n📊 Raw Data 동기화 상태 리포트")
        print("=" * 50)
        
        # raw_data 폴더 스캔
        raw_files_info = self.scan_raw_data_folder()
        sync_status = self.compare_with_db(db_manager)
        
        # raw_data 폴더 정보
        print(f"\n📁 Raw Data 폴더: {self.raw_data_path}")
        print(f"   총 파일 수: {len(raw_files_info)}개")
        
        if raw_files_info:
            total_size_mb = sum(info['size_mb'] for info in raw_files_info)
            print(f"   총 파일 크기: {total_size_mb:.2f} MB")
            
            # 파일 목록 테이블
            df_raw = pd.DataFrame(raw_files_info)
            print("\n📋 Raw Data 파일 목록:")
            display_cols = ['filename', 'extension', 'size_mb', 'modified_date']
            print(df_raw[display_cols].to_string(index=False))
        
        # DB 상태
        db_status = db_manager.get_status()
        print(f"\n🗄️ ChromaDB 상태:")
        print(f"   DB 존재: {'✅' if db_status['db_exists'] else '❌'}")
        print(f"   총 문서 수: {db_status['document_count']}개")
        print(f"   저장된 파일 수: {len(sync_status['all_db_files'])}개")
        
        # 동기화 상태
        print(f"\n🔄 동기화 상태:")
        print(f"   새 파일 (추가 필요): {len(sync_status['new_files'])}개")
        print(f"   기존 파일 (동기화됨): {len(sync_status['existing_files'])}개")
        print(f"   고아 파일 (DB에만 존재): {len(sync_status['orphaned_files'])}개")
        
        # 상세 정보
        if sync_status['new_files']:
            print(f"\n📥 추가할 새 파일들:")
            for filename in sync_status['new_files']:
                print(f"   - {filename}")
        
        if sync_status['orphaned_files']:
            print(f"\n👻 고아 파일들 (raw_data에 없음):")
            for filename in sync_status['orphaned_files']:
                print(f"   - {filename}")
        
        # 권장 사항
        print(f"\n💡 권장 사항:")
        if sync_status['new_files']:
            print(f"   🔄 sync_manager.sync_with_db(db_manager)를 실행하여 새 파일들을 추가하세요")
        if sync_status['orphaned_files']:
            print(f"   🧹 고아 파일들을 DB에서 제거하거나 원본 파일을 raw_data에 복원하는 것을 고려하세요")
        if not sync_status['new_files'] and not sync_status['orphaned_files']:
            print(f"   ✅ 모든 파일이 완벽하게 동기화되어 있습니다!")

print("✅ RawDataSyncManager 클래스가 정의되었습니다.")

✅ RawDataSyncManager 클래스가 정의되었습니다.


## 🚀 동기화 시스템 초기화 및 실행

이제 실제로 동기화를 실행해보겠습니다.

In [5]:
# 1. 관리자 초기화
print("🔧 관리자 초기화")

# ChromaDB 매니저 (실제 사용할 DB 경로)
db_manager = ChromaDBManager(
    db_path="./chroma_db",  # 실제 DB 경로
    embedding_model="text-embedding-3-large"
)

# Raw Data 동기화 매니저
sync_manager = RawDataSyncManager(
    raw_data_path="./raw_data"
)

print("\n✅ 관리자 초기화 완료!")

🔧 관리자 초기화
📁 DB 경로: ./chroma_db
🔤 임베딩 모델: text-embedding-3-large
📁 raw_data 폴더 경로: ./raw_data

✅ 관리자 초기화 완료!


In [8]:
# 2. 현재 동기화 상태 확인
print("📊 현재 동기화 상태 확인")
sync_manager.print_sync_report(db_manager)

📊 현재 동기화 상태 확인

📊 Raw Data 동기화 상태 리포트

📁 Raw Data 폴더: ./raw_data
   총 파일 수: 2개
   총 파일 크기: 9.60 MB

📋 Raw Data 파일 목록:
                       filename extension  size_mb       modified_date
       2025년 개정세법 해설 내지-12교.pdf      .pdf     8.86 2025-07-15 07:53:04
소득세법(법률)(제20615호)(20250701).pdf      .pdf     0.74 2025-07-15 04:28:14

🗄️ ChromaDB 상태:
   DB 존재: ✅
   총 문서 수: 1704개
   저장된 파일 수: 2개

🔄 동기화 상태:
   새 파일 (추가 필요): 0개
   기존 파일 (동기화됨): 2개
   고아 파일 (DB에만 존재): 0개

💡 권장 사항:
   ✅ 모든 파일이 완벽하게 동기화되어 있습니다!


In [7]:
# 3. 자동 동기화 실행
print("🔄 자동 동기화 시작")

# 청크 설정 (필요에 따라 조정)
CHUNK_SIZE = 1000      # 청크 크기
CHUNK_OVERLAP = 200    # 청크 겹침

# 동기화 실행
success = sync_manager.sync_with_db(
    db_manager=db_manager,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

if success:
    print("\n🎉 동기화가 성공적으로 완료되었습니다!")
else:
    print("\n❌ 동기화 중 문제가 발생했습니다.")

🔄 자동 동기화 시작
📝 2개의 새 파일을 DB에 추가합니다...
📥 기존 ChromaDB를 로드합니다...
✅ ChromaDB가 성공적으로 로드되었습니다! (852개 문서)

🔄 처리 중: 2025년 개정세법 해설 내지-12교.pdf
📄 파일 로드 성공: 2025년 개정세법 해설 내지-12교.pdf
📊 생성된 청크 수: 497
📝 기존 DB에 497개 문서를 추가합니다...
✅ 문서 추가 완료! (852 → 1349개)
   ✅ 2025년 개정세법 해설 내지-12교.pdf: 497개 청크 추가됨

🔄 처리 중: 소득세법(법률)(제20615호)(20250701).pdf
📄 파일 로드 성공: 소득세법(법률)(제20615호)(20250701).pdf
📊 생성된 청크 수: 355
📝 기존 DB에 355개 문서를 추가합니다...
✅ 문서 추가 완료! (1349 → 1704개)
   ✅ 소득세법(법률)(제20615호)(20250701).pdf: 355개 청크 추가됨

🎉 동기화 완료! 총 852개 청크가 추가되었습니다.

🎉 동기화가 성공적으로 완료되었습니다!


In [9]:
# 4. 동기화 후 상태 재확인
print("\n📊 동기화 후 최종 상태")
sync_manager.print_sync_report(db_manager)

# DB 상태 요약
status = db_manager.get_status()
print(f"\n📈 최종 DB 상태 요약:")
print(f"   총 문서 청크 수: {status['document_count']:,}개")
print(f"   DB 경로: {status['db_path']}")
print(f"   임베딩 모델: {status['embedding_model']}")


📊 동기화 후 최종 상태

📊 Raw Data 동기화 상태 리포트

📁 Raw Data 폴더: ./raw_data
   총 파일 수: 2개
   총 파일 크기: 9.60 MB

📋 Raw Data 파일 목록:
                       filename extension  size_mb       modified_date
       2025년 개정세법 해설 내지-12교.pdf      .pdf     8.86 2025-07-15 07:53:04
소득세법(법률)(제20615호)(20250701).pdf      .pdf     0.74 2025-07-15 04:28:14

🗄️ ChromaDB 상태:
   DB 존재: ✅
   총 문서 수: 1704개
   저장된 파일 수: 2개

🔄 동기화 상태:
   새 파일 (추가 필요): 0개
   기존 파일 (동기화됨): 2개
   고아 파일 (DB에만 존재): 0개

💡 권장 사항:
   ✅ 모든 파일이 완벽하게 동기화되어 있습니다!

📈 최종 DB 상태 요약:
   총 문서 청크 수: 1,704개
   DB 경로: ./chroma_db
   임베딩 모델: text-embedding-3-large


## 🔍 동기화된 문서 검색 테스트

동기화가 완료된 후 실제로 검색이 잘 작동하는지 테스트해보겠습니다.

In [10]:
# 5. 동기화된 문서들 검색 테스트
print("\n🔍 동기화된 문서 검색 테스트")

if db_manager.db:
    # 검색 테스트 쿼리들 (실제 문서에 맞게 수정하세요)
    test_queries = [
        "소득세",
        "개정세법",
        "세법 개정",
        "2025년"
    ]
    
    for query in test_queries:
        print(f"\n🔍 검색어: '{query}'")
        try:
            results = db_manager.db.similarity_search(query, k=3)
            
            if results:
                for i, result in enumerate(results, 1):
                    filename = os.path.basename(result.metadata.get('source', 'Unknown'))
                    page = result.metadata.get('page', 'N/A')
                    content_preview = result.page_content[:150] + "..." if len(result.page_content) > 150 else result.page_content
                    
                    print(f"   결과 {i}: {filename} (페이지 {page})")
                    print(f"   내용: {content_preview}")
                    print()
            else:
                print(f"   📭 '{query}'에 대한 검색 결과가 없습니다.")
        except Exception as e:
            print(f"   ❌ 검색 실패: {str(e)}")
else:
    print("❌ DB가 로드되지 않아 검색을 수행할 수 없습니다.")


🔍 동기화된 문서 검색 테스트

🔍 검색어: '소득세'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 34)
   내용: 소득세법(종합소득세 분야) ￭ 27
소득세법(종합소득세 분야)

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 34)
   내용: 소득세법(종합소득세 분야) ￭ 27
소득세법(종합소득세 분야)

   결과 3: 소득세법(법률)(제20615호)(20250701).pdf (페이지 0)
   내용: 법제처                                                            1                                                       국가법령정보센터
소득세법
 
소득세법
[시행 2025. ...


🔍 검색어: '개정세법'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 87)
   내용: 80 ￭ 2025년 개정세법 해설

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 87)
   내용: 80 ￭ 2025년 개정세법 해설

   결과 3: 2025년 개정세법 해설 내지-12교.pdf (페이지 33)
   내용: 26 ￭ 2025년 개정세법 해설


🔍 검색어: '세법 개정'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 121)
   내용: 114 ￭ 2025년 개정세법 해설

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 121)
   내용: 114 ￭ 2025년 개정세법 해설

   결과 3: 2025년 개정세법 해설 내지-12교.pdf (페이지 323)
   내용: 316 ￭ 2025년 개정세법 해설


🔍 검색어: '2025년'
   결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지 0)
   내용: 2025
20
2
5

   결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지 0)
   내용: 2

## 🎛️ 추가 관리 기능

필요에 따라 사용할 수 있는 추가 관리 기능들입니다.

In [11]:
# 특정 키워드로 검색하기 (사용자 정의)
search_query = "검색하고 싶은 키워드"  # 원하는 키워드로 변경

if db_manager.db:
    print(f"🔍 '{search_query}' 검색 결과:")
    try:
        results = db_manager.db.similarity_search(search_query, k=5)
        
        for i, result in enumerate(results, 1):
            filename = os.path.basename(result.metadata.get('source', 'Unknown'))
            page = result.metadata.get('page', 'N/A')
            score = result.metadata.get('score', 'N/A')  # 유사도 점수 (가능한 경우)
            
            print(f"\n📄 결과 {i}: {filename} (페이지: {page})")
            print(f"내용: {result.page_content[:200]}...")
    except Exception as e:
        print(f"❌ 검색 실패: {str(e)}")
else:
    print("❌ DB가 로드되지 않았습니다.")

🔍 '검색하고 싶은 키워드' 검색 결과:

📄 결과 1: 2025년 개정세법 해설 내지-12교.pdf (페이지: 161)
내용: 차례01. 정상가격 조정에 따른 경정청구 합리화· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·15702. 상계거래 적용대상 조문 정비· · · ·...

📄 결과 2: 2025년 개정세법 해설 내지-12교.pdf (페이지: 161)
내용: 차례01. 정상가격 조정에 따른 경정청구 합리화· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·15702. 상계거래 적용대상 조문 정비· · · ·...

📄 결과 3: 2025년 개정세법 해설 내지-12교.pdf (페이지: 32)
내용: 소득세법(종합소득세 분야)2025년 개정세법 해설
집 필 진
문의사항
징세법무국 법규과 행정사무관  노 영 인징세법무국 법규과 국세조사관  송 선 용징세법무국 법규과 국세조사관  김 성 희징세법무국 법규과 국세조사관  김 한 근징세법무국 법규과 국세조사관  박 광 춘
(044) 204-3117~21...

📄 결과 4: 2025년 개정세법 해설 내지-12교.pdf (페이지: 32)
내용: 소득세법(종합소득세 분야)2025년 개정세법 해설
집 필 진
문의사항
징세법무국 법규과 행정사무관  노 영 인징세법무국 법규과 국세조사관  송 선 용징세법무국 법규과 국세조사관  김 성 희징세법무국 법규과 국세조사관  김 한 근징세법무국 법규과 국세조사관  박 광 춘
(044) 204-3117~21...

📄 결과 5: 2025년 개정세법 해설 내지-12교.pdf

In [13]:
# DB 상세 정보 확인
print("📊 ChromaDB 상세 정보")
print("=" * 30)

status = db_manager.get_status()
files_in_db = db_manager.get_files_in_db()

print(f"DB 경로: {status['db_path']}")
print(f"임베딩 모델: {status['embedding_model']}")
print(f"DB 존재 여부: {'✅' if status['db_exists'] else '❌'}")
print(f"DB 로드 상태: {'✅' if status['db_loaded'] else '❌'}")
print(f"총 문서 청크 수: {status['document_count']:,}개")
print(f"저장된 파일 수: {len(files_in_db)}개")

if files_in_db:
    print("\n📋 저장된 파일 목록:")
    for i, filename in enumerate(files_in_db, 1):
        print(f"   {i}. {filename}")

📊 ChromaDB 상세 정보
DB 경로: ./chroma_db
임베딩 모델: text-embedding-3-large
DB 존재 여부: ✅
DB 로드 상태: ✅
총 문서 청크 수: 1,704개
저장된 파일 수: 2개

📋 저장된 파일 목록:
   1. 2025년 개정세법 해설 내지-12교.pdf
   2. 소득세법(법률)(제20615호)(20250701).pdf


## 🗑️ DB 관리 기능 (주의해서 사용)

⚠️ **주의**: 아래 기능들은 DB를 삭제하거나 초기화합니다. 신중하게 사용하세요.

In [ ]:
# DB 완전 삭제 (주의: 모든 데이터가 삭제됩니다)
# 실제 삭제를 원하면 아래 주석을 해제하고 실행하세요

# confirm_delete = input("정말로 DB를 삭제하시겠습니까? (yes/no): ")
# if confirm_delete.lower() == 'yes':
#     success = db_manager.delete_db()
#     if success:
#         print("✅ DB가 성공적으로 삭제되었습니다.")
#     else:
#         print("❌ DB 삭제에 실패했습니다.")
# else:
#     print("✋ DB 삭제가 취소되었습니다.")

print("💡 DB 삭제 기능은 주석 처리되어 있습니다. 필요시 주석을 해제하여 사용하세요.")

## 📚 사용 가이드 및 팁

### 🔄 정기적인 동기화 워크플로우:
1. **새 문서 추가**: `raw_data` 폴더에 .txt 또는 .pdf 파일 추가
2. **상태 확인**: `sync_manager.print_sync_report(db_manager)` 실행
3. **동기화 실행**: `sync_manager.sync_with_db(db_manager)` 실행
4. **검색 테스트**: 새로 추가된 문서가 검색되는지 확인

### ⚙️ 설정 조정:
- **청크 크기**: 문서 유형에 따라 `CHUNK_SIZE` 조정 (500-2000 권장)
- **청크 겹침**: 문맥 유지를 위해 `CHUNK_OVERLAP` 조정 (100-300 권장)
- **임베딩 모델**: 비용과 성능을 고려하여 모델 선택

### 🚨 주의사항:
- OpenAI API 사용량 모니터링 필요
- 대용량 파일 처리 시 시간이 오래 걸릴 수 있음
- 중요한 DB는 정기적으로 백업 권장

### 🎯 최적화 팁:
- 파일명에 의미 있는 이름 사용
- 관련 문서들을 함께 배치
- 정기적인 고아 파일 정리
- 검색 키워드 최적화